In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from model import Agent
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2024-06-19 08:42:52.685585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 08:42:52.892603: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-19 08:42:52.915065: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-19 08:42:52.915080: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
df = pd.read_csv('clean_health_data.csv')
df

,Billing_Amount,Age,admited_days,Gender_Male,Blood Type_A-,Blood Type_AB+,Blood Type_AB-,Blood Type_B+,Blood Type_B-,Blood Type_O+,...,Insurance Provider_Medicare,Insurance Provider_UnitedHealthcare,Admission Type_Emergency,Admission Type_Urgent,Medication_Ibuprofen,Medication_Lipitor,Medication_Paracetamol,Medication_Penicillin,Test Results_Inconclusive,Test Results_Normal
0,18856.281306,0.223684,0.034483,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
1,33643.327287,0.644737,0.172414,1,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
2,27955.096079,0.828947,0.482759,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,37909.782410,0.197368,1.000000,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
4,14238.317814,0.394737,0.655172,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55495,2650.714952,0.381579,1.000000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
55496,31457.797307,0.631579,0.275862,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
55497,27620.764717,0.328947,0.931034,0,0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0
55498,32451.092358,0.394737,0.172414,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [3]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.1, random_state =43 )
Y_train

18608    48735.848001
20123    42615.243851
9749      3995.424761
10325    26608.430185
27068    28733.379588
             ...     
18448    30896.731703
40753    28559.161643
51455    24378.326343
19776    29357.089704
14148    33211.426429
Name: Billing_Amount, Length: 49950, dtype: float64

In [5]:
def scale_to_minus1_1(X):
    X_min = np.min(X)
    X_max = np.max(X)
    return -1 + 2 * (X - X_min) / (X_max - X_min)

In [6]:
Y_train = scale_to_minus1_1(Y_train)
Y_train

18608    0.852904
20123    0.629413
9749    -0.780770
10325    0.044932
27068    0.122524
           ...   
18448    0.201518
40753    0.116162
51455   -0.036499
19776    0.145298
14148    0.286038
Name: Billing_Amount, Length: 49950, dtype: float64

In [7]:
agent = Agent(X.shape[1], 1)

2024-06-19 08:42:57.146200: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-19 08:42:57.147686: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-19 08:42:57.147743: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-06-19 08:42:57.147775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-06-19 08:42:57.147806: W tensorflow/c

In [8]:
def calculate_reward(pred, label):
    pred = np.array(pred)
    label = np.array(label)
    return - np.mean((pred-label)**2)

In [10]:
total_rewards = []
episodes = 2000
batch_size = 32

for episode in range(episodes):
    loss = 0 
    for i in range(0, len(X_train), batch_size):
        end = i + batch_size
        states = X_train[i:end]
        states = np.expand_dims(states, axis=2).astype('float32')
        labels = Y_train[i:end]

        actions = agent.get_action(states)
        
        reward = calculate_reward(labels,actions)

        next_states = states

        data = [states, actions, next_states, reward]
        loss += np.mean(reward)
        agent.train(data)

    if episode % 2 == 0:
        agent.update()
        print("Update!")

    print(f'Epoch {episode}, Average Loss: {-loss / len(X_train)}')



Update!
Epoch 0, Average Loss: 0.017544971071758182
Epoch 1, Average Loss: 0.01513143172877334
Update!
Epoch 2, Average Loss: 0.01114281380504472
Epoch 3, Average Loss: 0.010435896492808032
Update!
Epoch 4, Average Loss: 0.010560484608797915
Epoch 5, Average Loss: 0.01153793504910292
Update!
Epoch 6, Average Loss: 0.01149463866453942
Epoch 7, Average Loss: 0.012730825820795183


KeyboardInterrupt: 

array([[0.25      , 0.93103448, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.69736842, 0.17241379, 1.        , ..., 1.        , 1.        ,
        0.        ],
       [0.67105263, 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       ...,
       [0.85526316, 0.72413793, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.60526316, 0.24137931, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.65789474, 0.24137931, 1.        , ..., 0.        , 0.        ,
        1.        ]])

In [19]:
pred = agent.actor.model(np.array(X_test))

In [24]:
pred.numpy()

array([[-0.34866744],
       [-0.42665568],
       [ 0.4922311 ],
       ...,
       [-0.03887913],
       [-0.48883995],
       [-0.16072996]], dtype=float32)

In [28]:
Y_test = scale_to_minus1_1(Y_test).values

array([-0.55889092, -0.47336824, -0.76521637, ..., -0.3003652 ,
        0.5316906 ,  0.11724914])

In [37]:
np.mean((Y_test - pred.numpy().flatten())**2)

0.40265183557758255